In [41]:
import pandas as pd
import requests

In [42]:
def format_date(date):
    """
    Функция принимает на вход дату в формате день/месяц/год и возвращает в формате год-месяц-день
    
    Параметры:
        date: string — дата
    """
    
    day, month, year = date.split("/")
    
    return "{}-{}-{}".format(year, month, day)

In [43]:
def get_exchange_rates(date_start, date_end, save=False, filename=None):
    """
    Функция принимает на вход начало и конец периода, за которые нужно получить данные о курсе обмена
    хорватской куны на американский доллар.
    Возвращает DataFrame со столбцами [Date, Value]. Если указана опция save, то сохраняет созданный 
    DataFrame с указанным именем в формате .xlsx
    
    Параметры:
        date_start: string — начало периода,
        date_end: string — конец периода,
        save: boolean — сохранять данные в excel-файл или нет, по умолчанию save=False,
        filename: string — если save=True, то является обязательным, название excel-файла,
        web_driver: string — путь к драйверу Firefox
    """
    
    if save and not filename:
        raise ValueError("Filename should be specified when save option is True")
        
    # осуществляем запрос к сайту национального банка Хорватии с указанным промежутком дат
    request_string = "https://api.hnb.hr/tecajn/v1?datum-od={}&datum-do={}&valuta=USD".format(format_date(date_start), 
                                                                                              format_date(date_end))
    request = requests.get(request_string)
    
    # переводим полученный JSON в DataFrame
    exchange_rates_df = pd.read_json(request.text)
    
    # выбираем нужные столбцы из DataFrame
    exchange_rates_df = exchange_rates_df[['Datum primjene', 'Srednji za devize']]
    
    # форматируем DataFrame к нужному формату
    exchange_rates_df = exchange_rates_df.rename(columns={"Datum primjene": "Date", "Srednji za devize": "HRK"})
    exchange_rates_df['Date'] = pd.to_datetime(exchange_rates_df['Date'], format='%d.%m.%Y').dt.strftime('%d/%m/%Y')
    exchange_rates_df['HRK'] = exchange_rates_df['HRK'].str.replace(',', '.').astype(float)
    
    if save:
        exchange_rates_df.to_excel(filename, index=False)
        
    return exchange_rates_df

In [44]:
date_start = "10/10/2022"
date_end = "18/10/2022"

In [45]:
get_exchange_rates(date_start, date_end, save=True, filename="exchange_rates_hrk.xlsx")

,Date,HRK
0,10/10/2022,7.673986
1,11/10/2022,7.752944
2,12/10/2022,7.757543
3,13/10/2022,7.750544
4,14/10/2022,7.738007
5,15/10/2022,7.738317
6,16/10/2022,7.738317
7,17/10/2022,7.738317
8,18/10/2022,7.717326
